# Alternative 3 - End-to-end learning on raw features

In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf

In [2]:
df = pd.read_csv("./datasets/df_raw_features.tar.gz")
df.head()

,pid,time,id,sort,sleep_phase,act_0,act_1,act_2,act_3,act_4,...,hr_3,hr_4,hr_5,hr_6,hr_7,hr_8,hr_9,hr_10,hr_11,hr_12
0,1,29,"(1, 0)",0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0
1,1,59,"(1, 1)",1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0,75.0
2,1,89,"(1, 2)",2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0,75.0,76.0
3,1,119,"(1, 3)",3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,73.0,75.0,76.0,75.0
4,1,149,"(1, 4)",4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,73.0,75.0,76.0,75.0,80.0


In [3]:
df["sleep_phase"].unique()
# 0 -> Wake
# 1 -> phase 1 (light sleep)
# 2 -> phase 2 (deep sleep N1)
# 3 -> phase 3 (deep sleep N2)
# 4 -> NREM    ()
# 5 -> REM

array([0., 1., 2., 5., 3., 4.])

In [4]:
df["bin_sleep_phase"] = df["sleep_phase"] > 0

In [26]:
MASKING_VALUE = -1000

def generate_XY(df, maxdim, ycol="bin_sleep_phase"):

    hr_cols = dict([(int(k.split("_")[1]), k) for k in df.keys() if k.startswith("hr_")])
    last_hr_key = sorted(hr_cols.items(), key=lambda item: item[0])[-1][1]

    act_cols = dict([(int(k.split("_")[1]), k) for k in df.keys() if k.startswith("act_")])
    last_act_key = sorted(act_cols.items(), key=lambda item: item[0])[-1][1]

    hr = df[last_hr_key].values
    act = df[last_act_key].values
    
    Y  = df[ycol].astype(int).values.reshape(-1, 1)
    X = np.stack((act,hr))
    X = X.transpose(1,0)
    
    if maxdim > X.shape[0]:
        # Pad sequences (e.g., to the maxium length in the sequence or a constant like 8-10 hours)
        X = np.pad(X, ((0, maxdim-X.shape[0]), (0,0)), "constant", constant_values=MASKING_VALUE)
        Y = np.pad(Y, ((0, maxdim-Y.shape[0]), (0,0)), "constant", constant_values=-1)
    else:
        pass
        # Crop the sequence at some maxium length
        X = X[:maxdim]
        Y = Y[:maxdim]
        
    # Expand dims
    X = np.expand_dims(X, axis=0)
    Y = np.expand_dims(Y, axis=0)
    
    X = X.astype(float)
    Y = Y.astype(int)
    
    return X, Y
    

In [27]:
X, Y = generate_XY(df[df["pid"]==1], maxdim=800)

In [48]:
max_dimension = df.groupby("pid").size().max()
print("The largest dimension is %d" % (max_dimension))

quantile08 = int(df.groupby("pid").size().quantile(0.8))
print("Quantile 0.8 %d" % (quantile08))

dimension = max_dimension
dimension = 1200 # 1200 = 10 hours

The largest dimension is 1615
Quantile 0.8 1149


In [49]:
df_XY = df.groupby("pid").apply(lambda x: generate_XY(x, maxdim=dimension))
df_XY.head()

pid
1     ([[[ 0. 73.], [ 0. 75.], [ 0. 76.], [ 0. 75.],...
16    ([[[ 0. 67.], [ 0. 65.], [ 0. 66.], [ 0. 67.],...
21    ([[[11. 77.], [27. 63.], [ 0. 64.], [ 1. 65.],...
28    ([[[ 0. 72.], [ 0. 69.], [ 0. 69.], [ 0. 70.],...
33    ([[[ 0. 53.], [ 0. 52.], [ 0. 53.], [ 0. 51.],...
dtype: object

In [50]:
idx = 30
df_XY.iloc[idx][0].shape, df_XY.iloc[idx][1].shape

((1, 1200, 2), (1, 1200, 1))

In [51]:
df_XY.iloc[idx][0]

array([[[    2.,    70.],
        [    0.,    68.],
        [    0.,    70.],
        ...,
        [-1000., -1000.],
        [-1000., -1000.],
        [-1000., -1000.]]])

In [52]:
df_XY.iloc[idx][1]

array([[[ 1],
        [ 1],
        [ 1],
        ...,
        [-1],
        [-1],
        [-1]]])

In [53]:
xs, ys = [], []
for row_id, (x, y) in df_XY.items():
    xs.append(x)
    ys.append(y)
    
xs = np.array(xs, dtype=object)
ys = np.array(ys, dtype=object)


In [54]:
subjects_train_idx = range(100)
X_train = np.vstack(xs[subjects_train_idx])
Y_train = np.vstack(ys[subjects_train_idx])

subjects_test_idx = range(100, 150)
X_val = np.vstack(xs[subjects_test_idx])
Y_val = np.vstack(ys[subjects_test_idx])

subjects_test_idx = range(150, 200)
X_test = np.vstack(xs[subjects_test_idx])
Y_test = np.vstack(ys[subjects_test_idx])


In [77]:
def cnn_lstm_model(cnn_d=32, lstm_d=32):
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Masking(mask_value=MASKING_VALUE))
    
#     model.add(tf.keras.layers.Conv1D(cnn_d, kernel_size=(3,), padding='same'))    
#     model.add(tf.keras.layers.Activation(tf.nn.relu))
    
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.LSTM(lstm_d, return_sequences=True))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.LSTM(lstm_d, return_sequences=True))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.LSTM(lstm_d, return_sequences=True))
    
    # model.add(tf.keras.layers.Conv1D(1, 1, activation='sigmoid', padding='same'))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid", name='output'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

cnnlstm_model = cnn_lstm_model()

In [78]:
with tf.device('/cpu:0'):
    early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    cnnlstm_model.fit(X_train.astype(float), Y_train.astype(int),
                      validation_data=(X_val.astype(float), Y_val.astype(int)), 
                      epochs=50, 
                      batch_size=8,
                      callbacks=[early_stop_callback])

Epoch 1/50
13/13 [==============================] - 28s 2s/step - loss: 0.5105 - accuracy: 0.6958 - val_loss: 0.4549 - val_accuracy: 0.7415
Epoch 2/50
13/13 [==============================] - 17s 1s/step - loss: 0.4249 - accuracy: 0.7660 - val_loss: 0.5446 - val_accuracy: 0.4736
Epoch 3/50
13/13 [==============================] - 17s 1s/step - loss: 0.3893 - accuracy: 0.8003 - val_loss: 0.6028 - val_accuracy: 0.4406
Epoch 4/50
13/13 [==============================] - 17s 1s/step - loss: 0.3783 - accuracy: 0.8051 - val_loss: 0.5027 - val_accuracy: 0.6716


In [74]:
# def cnn_lstm_model2(dimension, cnn_d=10, lstm_d=4):
    
#     input_size = (dimension, 2)
#     input_layer = tf.keras.layers.Input(shape=input_size)
#     masked_layer = tf.keras.layers.Masking(mask_value=MASKING_VALUE, input_shape=input_size)(input_layer)
        
#     # masked_layer = tf.keras.layers.BatchNormalization(epsilon=1e-03, axis=-1, momentum=0.99)(masked_layer)
    
#     layer1 = tf.keras.layers.Conv1D(cnn_d, kernel_size=3, padding='same', activation='softplus')(masked_layer)
#     layer2 = tf.keras.layers.Conv1D(cnn_d, kernel_size=5, padding='same', activation='softplus')(masked_layer)
# #     layer3 = tf.keras.layers.Conv1D(cnn_d, kernel_size=7, padding='same', activation='softplus')(masked_layer)
# #     layer4 = tf.keras.layers.Conv1D(cnn_d, kernel_size=30, padding='same', activation='softplus')(masked_layer)
# #     layer5 = tf.keras.layers.Conv1D(cnn_d, kernel_size=60, padding='same', activation='softplus')(masked_layer)
# #     layer6 = tf.keras.layers.Conv1D(cnn_d, kernel_size=120, padding='same', activation='softplus')(masked_layer)
# #     layer7 = tf.keras.layers.Conv1D(cnn_d, kernel_size=240, padding='same', activation='softplus')(masked_layer)
 
# #     concat = tf.keras.layers.Concatenate(axis=2)([layer1, layer2, layer3, layer4, layer5, layer6, layer7])
#     concat = tf.keras.layers.Concatenate(axis=2)([layer1, layer2])
    
#     x = tf.keras.layers.Dropout(0.1)(concat)
#     x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, mask=masked_layer))(x)
# #     x = tf.keras.layers.Dropout(0.5)(x)
# #     x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True))(x)
# #     x = tf.keras.layers.Dropout(0.5)(x)
# #     x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)

#     output_stages = tf.keras.layers.Conv1D(1, 1, activation='sigmoid', padding='same', name="stages")(x)
#     model = tf.keras.models.Model(inputs=input_layer, outputs=output_stages)

#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        
#     return model

# cnnlstm_model = cnn_lstm_model2(dimension)

In [75]:
with tf.device('/cpu:0'):
    p = cnnlstm_model.predict(x=X_test.astype(float))

2/2 [==============================] - 1s 77ms/step


In [76]:
with tf.device('/cpu:0'):
    cnnlstm_model.evaluate(x=X_test.astype(float), y=Y_test.astype(int))

2/2 [==============================] - 0s 75ms/step - loss: 0.5372 - accuracy: 0.2707


In [71]:
from sklearn import metrics

y0 = Y_test[0]
y_filter = (y0 != MASKING_VALUE)

metrics.accuracy_score(p[0][y_filter] > 0.5, y0[y_filter].astype(int))


0.5433333333333333

In [ ]:

        input_size1, input_size2, input_size3, output_size = (1200, 30, 1), (1200, 1), (1200, 136, 1), 3

        input_layer1 = tf.keras.layers.Input(shape=input_size1, name="input_1")
        masked_layer1 = tf.keras.layers.Masking(mask_value=3000, input_shape=input_size1)(input_layer1)
        if bn_input:
            masked_layer1 = tf.keras.layers.BatchNormalization(epsilon=1e-03, axis=-1, momentum=0.99)(masked_layer1)
        masked_layer1 = tf.squeeze(masked_layer1, axis=3)

        input_layer2 = tf.keras.layers.Input(shape=input_size2, name="input_2")
        masked_layer2 = tf.keras.layers.Masking(mask_value=3000, input_shape=input_size2)(input_layer2)
        if bn_input:
            masked_layer2 = tf.keras.layers.BatchNormalization(epsilon=1e-03, axis=-1, momentum=0.99)(masked_layer2)


        layer1_1 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=3, padding='same', activation='softplus')(masked_layer1)
        layer1_2 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=5, padding='same', activation='softplus')(masked_layer1)
        layer1_3 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=7, padding='same', activation='softplus')(masked_layer1)
        layer1_4 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=30, padding='same', activation='softplus')(masked_layer1)
        layer1_5 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=60, padding='same', activation='softplus')(masked_layer1)
        layer1_6 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=120, padding='same', activation='softplus')(masked_layer1)
        layer1_7 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=240, padding='same', activation='softplus')(masked_layer1)
        layer1_8 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=3, padding='same', activation='relu')(masked_layer1)
        layer1_9 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=5, padding='same', activation='relu')(masked_layer1)
        layer1_10 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=7, padding='same', activation=None)(masked_layer1)
        layer1_10 = tf.keras.layers.ThresholdedReLU(theta=1.0)(layer1_10)
        layer1_11 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=5, padding='same', activation='tanh')(masked_layer1)
        layer1_12 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=30, padding='same', activation='tanh')(masked_layer1)

        layer2_1 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=3, padding='same', activation='softplus')(masked_layer2)
        layer2_2 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=5, padding='same', activation='softplus')(masked_layer2)
        layer2_3 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=7, padding='same', activation='softplus')(masked_layer2)
        layer2_4 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=30, padding='same', activation='softplus')(masked_layer2)
        layer2_5 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=60, padding='same', activation='softplus')(masked_layer2)
        layer2_6 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=120, padding='same', activation='softplus')(masked_layer2)
        layer2_7 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=240, padding='same', activation='softplus')(masked_layer2)
        layer2_8 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=3, padding='same', activation='relu')(masked_layer2)
        layer2_9 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=5, padding='same', activation='relu')(masked_layer2)
        layer2_10 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=7, padding='same', activation=None)(masked_layer2)
        layer2_10 = tf.keras.layers.ThresholdedReLU(theta=1.0)(layer2_10)
        layer2_11 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=5, padding='same', activation='tanh')(masked_layer2)
        layer2_12 = tf.keras.layers.Conv1D(cnn_dim, kernel_size=30, padding='same', activation='tanh')(masked_layer2)

        if concatenation == "simple":
            concat = tf.keras.layers.Concatenate(axis=2)(
                [layer1_1, layer1_2, layer1_3, layer1_4, layer1_5, layer1_6, layer1_7, layer1_8, layer1_9, layer1_10,
                 layer1_11, layer1_12, layer2_1, layer2_2, layer2_3, layer2_4, layer2_5, layer2_6, layer2_7, layer2_8,
                 layer2_9, layer2_10, layer2_11, layer2_12])
        elif concatenation == "pairs":
            concat = tf.keras.layers.Concatenate(axis=2)(
                [layer1_1 + layer2_1, layer1_2 + layer2_2, layer1_3 + layer2_3, layer1_4 + layer2_4,
                 layer1_5 + layer2_5, layer1_6 + layer2_6, layer1_7 + layer2_7, layer1_8 + layer2_8,
                 layer1_9 + layer2_9, layer1_10 + layer2_10, layer1_11 + layer2_11, layer1_12 + layer2_12])

        elif concatenation == "all":
            concat = layer1_1 + layer2_1 + layer1_2 + layer2_2 + layer1_3 + layer2_3 + layer1_4 + layer2_4 + layer1_5 + layer2_5 + \
                     layer1_6 + layer2_6 + layer1_7 + layer2_7 + layer1_8 + layer2_8 + layer1_9 + layer2_9 + layer1_10 + layer2_10 + \
                     layer2_11 + layer2_12 + layer1_11 + layer1_12

        x = tf.keras.layers.Dropout(cnn_concat_dropout)(concat)
        x = tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True))(x)
        x = tf.keras.layers.Dropout(lstm_dropout)(x)
        x = tf.keras.layers.Bidirectional(LSTM(32, return_sequences=True))(x)
        x = tf.keras.layers.Dropout(lstm_dropout)(x)
        x = tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True))(x)

        output_stages = tf.keras.layers.Conv1D(output_size, 1, activation='softmax', padding='same', name="stages")(x)

        if multiple_outputs:
            output_idx0 = tf.keras.layers.Conv1D(2, 1, activation='softmax', padding='same', name="idx0")(x)
            output_idx1 = tf.keras.layers.Conv1D(2, 1, activation='softmax', padding='same', name="idx1")(x)
            output_idx2 = tf.keras.layers.Conv1D(2, 1, activation='softmax', padding='same', name="idx2")(x)
            model = tf.keras.models.Model(inputs=[input_layer1, input_layer2],
                                          outputs=[output_stages, output_idx0, output_idx1, output_idx2])

        else:
            model = tf.keras.models.Model(inputs=[input_layer1, input_layer2], outputs=[output_stages])

        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=16000, decay_rate=0.98,
                                                                     staircase=True)

        def matthews_correlation_coefficient(y_true, y_pred):
            tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
            tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
            fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
            fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))

            num = tp * tn - fp * fn
            den = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
            return num / K.sqrt(den + K.epsilon())

        if multiple_outputs:
            model.compile(optimizer=tfa.optimizers.AdamW(lr_schedule),
                          loss=[tf.keras.losses.CategoricalCrossentropy(),
                                tf.keras.losses.BinaryCrossentropy(),
                                tf.keras.losses.BinaryCrossentropy(),
                                tf.keras.losses.BinaryCrossentropy()
                                ],
                          metrics={"stages": ['categorical_accuracy', matthews_correlation_coefficient],
                                   "idx0": ['binary_accuracy', matthews_correlation_coefficient],
                                   "idx1": ['binary_accuracy', matthews_correlation_coefficient],
                                   "idx2": ['binary_accuracy', matthews_correlation_coefficient],
                                   },
                          loss_weights={"stages": 1.0, "idx0": 1., "idx1": 0.5, "idx2": 0.5},
                          )
        else:
            model.compile(optimizer=tfa.optimizers.AdamW(lr_schedule),
                          loss=tfa.losses.SigmoidFocalCrossEntropy(),
                          metrics={"stages": ['categorical_accuracy', matthews_correlation_coefficient]}
                          )

        model.summary()
        return model